In [7]:
import numpy as np
import matplotlib.pyplot as plt

In [10]:
poly = np.polyfit(np.linspace(1, 10, 100), np.linspace(1, 10, 100)**2, 3); poly

array([ 1.57143202e-16,  1.00000000e+00,  2.14540031e-14, -2.55795385e-14])

In [11]:
np.poly1d(poly)

poly1d([ 1.57143202e-16,  1.00000000e+00,  2.14540031e-14, -2.55795385e-14])

In [ ]:

def powers_of_X(X, degree):
    powers = np.arange(degree + 1).reshape(1, -1)
    return X ** powers
	

In [ ]:
def poly_fit(data, degree, alpha):
    X = powers_of_X(data[0], degree)  # Matrix N x d
    Y = data[1].reshape(-1, 1)  # Matrix N x 1
    #
    # TODO: implement the closed-form solution for Theta
    #
    # Please note that np.linalg.inv may be numerically unstable.
    # It is better to use np.linalg.solve or even a QR decomposition.
    #
    
    I = np.identity(degree + 1)
    left = (X.T @ X) + I*alpha
    right = X.T @ Y
    Theta = np.linalg.solve(left, right)
    return Theta.T[0]